In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook

In [6]:
url = 'https://apps-congress.eular.org/eular2020/en-GB/PublicProgram/SessionTableJson?'
params = {'pDay':'2020-06-04T06:00:00','pEventId':'7','pProgramGrade':'0','pShowPrivate':'false'}
data = requests.get(url,params = params)

In [8]:
dict = data.json()


In [32]:
dict['Sessions'][1]

{'Id': 574,
 'Name': 'WIN - IgG4 disease',
 'TextWithFormat': 'WIN - IgG4 disease',
 'StartDateTime': '/Date(1591258500000)/',
 'EndDateTime': '/Date(1591260900000)/',
 'Number': '',
 'Short': '',
 'Description': '&lt;ul&gt;&lt;li&gt;To receive an update on IgG4 disease.&lt;/li&gt;&lt;/ul&gt;',
 'Color': '#002060',
 'ViewCounter': 14,
 'IsPoster': False,
 'RoomId': 78,
 'RoomName': 'Festhalle',
 'RoomSort': 1,
 'SessionTypeName': 'WIN session',
 'SessionGroupNames': '',
 'IsInPersonalProgram': False,
 'SessionGroupColors': None,
 'Tags': [],
 'StartTimeString': '10:15',
 'EndTimeString': '10:55',
 'Chairs': None,
 'RoomIndex': 0,
 'StartTimeIndex': 1,
 'DayStart': '/Date(1591251300000)/',
 'TotalMinutesOfDay': 645,
 'Icons': None,
 'SessionGroupList': None,
 'SessionGroupColorList': None,
 'Text': 'WIN - IgG4 disease'}

In [14]:
for i in dict['Sessions']:
    print(i['Id'])

666
574
687
702
697
686
689
690
669
750
665
683
675
682
576
575
580
582
584
586
588
591
602
596
594
600
604
598
579
578
581
583
585
587
589
592
632
597
603
601
605
599
553
577
593


In [94]:
id = dict['Sessions'][1]['Id']


In [95]:
sessionUrl = 'https://apps-congress.eular.org/eular2020/en-GB/ProgramSearch/Program/?searchstring=&programid='+str(id)+'&pProgramGrade=All&pHideLogin=False&pHidePersonal=False'

In [22]:
url

'https://apps-congress.eular.org/eular2020/en-GB/PublicProgram/SessionTableJson?'

In [24]:
import pandas as pd

In [26]:
sessionUrl

'https://apps-congress.eular.org/eular2020/en-GB/ProgramSearch/Program/?searchstring=&programid=574&pProgramGrade=All&pHideLogin=False&pHidePersonal=False'

In [96]:
sessionData = requests.get(sessionUrl)
# sessionData = BeautifulSoup(sessionData.text,'lxml')
# sessionDataTable = sessionData.find_all('table',class_ = 'Result table table-striped programresult')[0]
dfList = pd.read_html(sessionData.text)
# sessionData.text
dfList[-1]

,Unnamed: 0,Title,Time,Appointment,Unnamed: 4,Unnamed: 5
0,NaN,What is new in IgG4 disease Dr. John H. Stone...,04 June 2020 10:20 - 11:00,NaN,NaN,NaN


In [98]:
df = dfList[-1].copy()
df['Room'] = dict['Sessions'][1]['RoomName']
df['Session Name'] = dict['Sessions'][1]['Name']
df['Description'] = dict['Sessions'][1]['Description'].strip().replace('&lt','').replace('/li','').replace('&gt','').replace(';','').replace('ul','').replace('li','').replace('/','')
df.drop(columns = ['Unnamed: 0','Unnamed: 4','Unnamed: 5','Appointment'],axis=1,inplace = True)

In [99]:
df

,Title,Time,Room,Session Name,Description
0,What is new in IgG4 disease Dr. John H. Stone...,04 June 2020 10:20 - 11:00,Festhalle,WIN - IgG4 disease,To receive an update on IgG4 disease.


In [14]:
dates = [3,4,5,6]
url = 'https://apps-congress.eular.org/eular2020/en-GB/PublicProgram/SessionTableJson?'
Masterdf = pd.DataFrame()
for i in tqdm_notebook(dates):
    pDay = '2020-06-0' + str(i) + 'T06:00:00'
    params = {'pDay': pDay,'pEventId':'7','pProgramGrade':'0','pShowPrivate':'false'}
    data = requests.get(url,params = params)
    dict_ = data.json()
    for j in tqdm_notebook(dict_['Sessions']):
        id = j['Id']
        sessionUrl = 'https://apps-congress.eular.org/eular2020/en-GB/ProgramSearch/Program/?searchstring=&programid='+str(id)+'&pProgramGrade=All&pHideLogin=False&pHidePersonal=False'
        sessionData = requests.get(sessionUrl)
        try:
            dfList = pd.read_html(sessionData.text)
            df = dfList[-1].copy()
            df['Room'] = j['RoomName']
            df['Session Name'] = j['Name']
            df['Chairs'] = j['Chairs']
            df['Description'] = j['Description']
            df.drop(columns = ['Unnamed: 0','Unnamed: 4','Unnamed: 5','Appointment'],axis=1,inplace = True)
            Masterdf = Masterdf.append(df,ignore_index = True)
        except:
            print('Yes')
#           df = pd.DataFrame()
            dict_1 = {}    
            dict_1['Room'] = j['RoomName']
            dict_1['Session Name'] = j['Name']
            dict_1['Chairs'] = j['Chairs']
            dict_1['Description'] = j['Description']
#             print(df)
            Masterdf = Masterdf.append(dict_1,ignore_index = True)
        

Yes



Yes
Yes




In [8]:
Masterdf.to_excel('Eular2020.xls')


In [15]:
Masterdf

,Chairs,Description,Room,Session Name,Time,Title
0,None,<ul><li>To be able to discuss the key activiti...,Festhalle,Opening plenary session,NaN,NaN
1,None,<ul><li>To understand the increased fracture r...,Festhalle,WIN - Bone fragility in chronic kidney disease,03 June 2020 14:20 - 14:55,What is new - Bone disease and fracture risk i...
2,Prof. Ernest Choy<br>Prof. Andrea Rubbert-Roth,To get an up-to-date information on oral thera...,Hall 3A,tsDMARD therapy of rheumatoid arthritis,03 June 2020 14:20 - 14:45,JAKi in the news – side effects MD Roy Fleisc...
3,Prof. Ernest Choy<br>Prof. Andrea Rubbert-Roth,To get an up-to-date information on oral thera...,Hall 3A,tsDMARD therapy of rheumatoid arthritis,03 June 2020 14:45 - 15:10,“Selective” JAKi on the verve – implications f...
4,Prof. Ernest Choy<br>Prof. Andrea Rubbert-Roth,To get an up-to-date information on oral thera...,Hall 3A,tsDMARD therapy of rheumatoid arthritis,03 June 2020 15:10 - 15:35,Ten years of JAKi in RA – Real life data from ...
...,...,...,...,...,...,...
686,Ms. Agnes Kocher<br>Dr. Kirsten Hoeper,&lt;ul&gt;&lt;li&gt;Take home messages for HPR...,Illusion,HPR Highlight Session,06 June 2020 13:50 - 14:05,"MD highlights Prof. Dr. Espen A Haavardsholm,..."
687,Ms. Agnes Kocher<br>Dr. Kirsten Hoeper,&lt;ul&gt;&lt;li&gt;Take home messages for HPR...,Illusion,HPR Highlight Session,06 June 2020 14:05 - 14:20,PARE representative PARE programme Mrs. Boria...
688,Ms. Agnes Kocher<br>Dr. Kirsten Hoeper,&lt;ul&gt;&lt;li&gt;Take home messages for HPR...,Illusion,HPR Highlight Session,06 June 2020 14:20 - 14:35,"HPR highlights Dr. Mwidimi Ndosi, United Kingdom"
689,(Speaker to be announced)<br>(Speaker to be an...,"&lt;ul&gt;&lt;li&gt;&lt;span lang=""EN-US""&gt;T...",Hall 3C,PARE highlights session,06 June 2020 13:50 - 14:15,Highlights from the Scientific Programme Dr. ...


In [16]:
Masterdf2 = Masterdf.copy()

In [17]:
Masterdf['Date'] = Masterdf['Time'].str.split('2020',expand = True)[0]

In [18]:
Masterdf['Start Time'] = Masterdf['Time'].str.split('2020',expand = True)[1].str.split(' - ',expand = True)[0]
Masterdf['End Time'] = Masterdf['Time'].str.split('2020',expand = True)[1].str.split(' - ',expand = True)[1]

In [19]:
Masterdf.columns

Index(['Chairs', 'Description', 'Room', 'Session Name', 'Time', 'Title',
       'Date', 'Start Time', 'End Time'],
      dtype='object')

In [20]:
cols = ['Session Name','Room','Description', 'Chairs', 'Time','Date', 'Title', 
       'Start Time', 'End Time']
Masterdf = Masterdf[cols]

In [46]:
Masterdf['Description'] = Masterdf2['Description'].str.strip().str.replace('&lt;ul&gt;&lt;li&gt;','').str.replace('&lt;/li&gt;&lt;/ul&gt;','').str.replace('ul type="disc"','').str.replace('&lt','').str.replace('/li','').str.replace('&gt','').str.replace(';','').str.replace('<ul>','').str.replace('<li>','').str.replace('/','').str.replace('<p>','').str.replace('<','').str.replace('span lang="EN-US"','').str.replace('span','').str.replace('>','').str.replace('nbsp','').str.replace('&','').str.replace('middot','').str.replace('lang="EN-GB"','')
Masterdf['Chairs'] = Masterdf['Chairs'].str.replace('<br>',',')

In [43]:
Masterdf['Description'][686]

'Take home messages for HPR after the EULAR Frankfurt 2020 Congress.'

In [44]:
Masterdf['Description']

0      To be able to discuss the key activities of EU...
1      To understand the increased fracture risk in p...
2      To get an up-to-date information on oral thera...
3      To get an up-to-date information on oral thera...
4      To get an up-to-date information on oral thera...
                             ...                        
686    Take home messages for HPR after the EULAR Fra...
687    Take home messages for HPR after the EULAR Fra...
688    Take home messages for HPR after the EULAR Fra...
689    To provide an overview of learning and outcome...
690    To provide an overview of learning and outcome...
Name: Description, Length: 691, dtype: object

In [47]:
Masterdf.to_excel('Eular2020.xls')